<a href="https://colab.research.google.com/github/hieutrgvu/text-generation-and-correction/blob/main/language-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **0. Running from Google Colab**

In [2]:
!git clone https://github.com/hieutrgvu/text-generation-and-correction.git

Cloning into 'text-generation-and-correction'...
remote: Enumerating objects: 1042, done.
remote: Counting objects: 100% (1042/1042), done.
remote: Compressing objects: 100% (1010/1010), done.
remote: Total 1042 (delta 26), reused 1031 (delta 22), pack-reused 0
Receiving objects: 100% (1042/1042), 6.01 MiB | 15.04 MiB/s, done.
Resolving deltas: 100% (26/26), done.


In [3]:
cd "text-generation-and-correction"

/content/text-generation-and-correction


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **1. Import**

In [5]:
import os
import random
import re
import numpy as np
import tensorflow as tf
import time
from scipy import special

# **2. Load, Clean and Augment Data**

In [95]:
# load
lines = []
data_dir = "./tiki-data"
for file in os.listdir(data_dir):
  if file.startswith("sach-"):
    with open(data_dir+"/"+file) as f:
      lines.extend(f.readlines())

print("Number of lines: ", len(lines))
lines[:10]

Number of lines:  10079


['"Madame Chic - Rất Thần Thái, Rất Paris"\n',
 'Hành Trình Của Linh Hồn\n',
 'Thai Giáo Theo Chuyên Gia - 280 Ngày - Mỗi Ngày Đọc Một Trang\n',
 'EAT CLEAN Thực Đơn 14 Ngày Thanh Lọc Cơ Thể Và Giảm Cân\n',
 'Thánh Kinh Dưỡng Da\n',
 '"Green Smoothies - Giảm Cân, Làm Đẹp Da, Tăng Cường Sức Đề Kháng Với 7 Ngày Uống Sinh Tố Xanh"\n',
 'BREW - Tuyệt Đỉnh Cà Phê Tại Nhà\n',
 'Khởi Sự Ăn Chay\n',
 'Đừng Chỉ Mặc Màu Đen\n',
 'Chào Juice\n']

In [98]:
# clean
bos = "{"
eos = "}"
regex = "[^0-9a-zạảãàáâậầấẩẫăắằặẳẵóòọõỏôộổỗồốơờớợởỡéèẻẹẽêếềệểễúùụủũưựữửừứíìịỉĩýỳỷỵỹđ]"
for i in range(len(lines)):
  lines[i] = re.sub(regex, " ", lines[i].lower()).strip()
  lines[i] = bos + re.sub(' +', ' ', lines[i])  + eos
lines[:10]

['{madame chic rất thần thái rất paris}',
 '{hành trình của linh hồn}',
 '{thai giáo theo chuyên gia 280 ngày mỗi ngày đọc một trang}',
 '{eat clean thực đơn 14 ngày thanh lọc cơ thể và giảm cân}',
 '{thánh kinh dưỡng da}',
 '{green smoothies gia m cân la m đe p da tăng cươ ng sư c đê kha ng vơ i 7 nga y uô ng sinh tô xanh}',
 '{brew tuyê t đi nh ca phê ta i nha}',
 '{khởi sự ăn chay}',
 '{đừng chỉ mặc màu đen}',
 '{chào juice}']

In [101]:
# augment
text = []
for line in lines:
  line = [line]*10
  text.extend(line)
random.shuffle(text)
text = "".join(text)
text[:500]

'{vietmath cùng con giỏi tư duy toán học tập 1}{rich habits thói quen thành công của những triệu phú tự thân}{giải mật ngoại hạng anh}{đời sống bí ẩn của cây}{sự giàu và nghèo của các dân tộc}{brew tuyê t đi nh ca phê ta i nha}{từ chiến lược marketing đến doanh nghiệp thành công}{science encyclopedia bách khoa thư về khoa học trái đất và vũ trụ}{triệu phú thức tỉnh bí kíp để khơi dòng suối nguồn thịnh vượng trong tâm thức}{bạn đắt giá bao nhiêu tặng kèm bộ bookmark tiki love books}{thị dân 3 0}{k'

In [225]:
#Create vocabulary 
vocab = sorted(set(text))
print("vocab len:", len(vocab))
#create an index for each character
char2idx = {u:i for i,u in enumerate(vocab)}
idx2char = np.array(vocab)
conver_text_to_int = np.array([char2idx[char] for char in text])

vocab len: 106


In [10]:
#convert the text vector into a stream of character indices.
char_dataset = tf.data.Dataset.from_tensor_slices(conver_text_to_int)
#Each sample has 100 chars
seq_length = 100
#convert char to sentences of 100 chars
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
#split into input and targer, each length 100
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [11]:
#shuffle and batch samples
BATCH_SIZE =30
dataset = dataset.shuffle(10000).batch(BATCH_SIZE,drop_remainder=True) 
embedding_dim = 256
rnn_units=1024

# **3. Model**

In [12]:
def build_model(embedding_dim,rnn_units,batch_size,vocab_size):
  model = tf.keras.Sequential(
  [tf.keras.layers.Embedding(vocab_size,embedding_dim,batch_input_shape=[batch_size,None]),
     tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)  
  ])
  return model

def build_lstm_model(embedding_dim,rnn_units,batch_size,vocab_size):
  model = tf.keras.Sequential([tf.keras.layers.Embedding(vocab_size,embedding_dim,batch_input_shape=[batch_size,None]),
     tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)  
    ])
  return model
  

def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

## **3.1. GRU**

In [ ]:
#Train model GRU layer
model = build_model(embedding_dim,rnn_units,BATCH_SIZE,len(vocab))
model.summary()
model_save_dir = '/content/drive/MyDrive/LSTM/RNN'
checkpoint_prefix = os.path.join(model_save_dir, "ckpt_{epoch}")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)
early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
model.compile(optimizer='adam', loss=loss)
history = model.fit(dataset, epochs=30,callbacks=[checkpoint_callback, early_stop_callback])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (30, None, 256)           27136     
_________________________________________________________________
gru_2 (GRU)                  (30, None, 1024)          3938304   
_________________________________________________________________
dense_2 (Dense)              (30, None, 106)           108650    
Total params: 4,074,090
Trainable params: 4,074,090
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
1583/1583 [==============================] - 49s 31ms/step - loss: 1.2216
Epoch 2/30
1583/1583 [==============================] - 49s 31ms/step - loss: 0.5236
Epoch 3/30
1583/1583 [==============================] - 49s 31ms/step - loss: 0.4420
Epoch 4/30
1583/1583 [==============================] - 49s 31ms/step - loss: 0.4282
Epoch 5/30
1583/1583 [============

In [13]:
model_save_dir = '/content/drive/MyDrive/LSTM/RNN'
generate_model = build_model(embedding_dim,rnn_units,1,len(vocab))
generate_model.load_weights(tf.train.latest_checkpoint(model_save_dir))

## **3.2. LSTM**

In [ ]:
#Train model LSTM 
model_lstm = build_lstm_model(embedding_dim,rnn_units,BATCH_SIZE,len(vocab))
model_lstm.summary()
#train model 
#add checkpoint save
model_save_dir = '/content/drive/My Drive/ML/RNN/checkpointlstm1'
checkpoint_prefix = os.path.join(model_save_dir, "ckpt_{epoch}")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)
model_lstm.compile(optimizer='adam', loss=loss)
model_lstm.fit(dataset, epochs=30,callbacks=[checkpoint_callback, early_stop_callback])

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (30, None, 256)           27136     
_________________________________________________________________
lstm_5 (LSTM)                (30, None, 1024)          5246976   
_________________________________________________________________
dense_20 (Dense)             (30, None, 106)           108650    
Total params: 5,382,762
Trainable params: 5,382,762
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
1583/1583 [==============================] - 59s 37ms/step - loss: 1.1461
Epoch 2/30
1583/1583 [==============================] - 59s 37ms/step - loss: 0.5018
Epoch 3/30
1583/1583 [==============================] - 59s 37ms/step - loss: 0.4193
Epoch 4/30
1583/1583 [==============================] - 59s 37ms/step - loss: 0.3967
Epoch 5/30
1583/1583 [===========

In [14]:
model_save_dir = '/content/drive/My Drive/ML/RNN/checkpointlstm1'
generate_model_lstm = build_lstm_model(embedding_dim,rnn_units,1,len(vocab))
generate_model_lstm.load_weights(tf.train.latest_checkpoint(model_save_dir)).expect_partial()

# **4. Text Generation**

In [15]:
def generate_text(model, start_string):
    num_generate = 100
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    print(input_eval.shape)
    text_generated = []
    model.reset_states() #delete hidden state

    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)# drop batch dimensionality
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        prob = special.softmax(predictions[-1])
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])
        if idx2char[predicted_id] == "}":
          text_generated = text_generated[:-1]
          break
        if max(prob) < 0.2:
          break
    return (start_string + ''.join(text_generated))

## **4.1. GRU**

In [60]:
#Build new model to generate
result_of_gru_char = generate_text(generate_model, start_string=u"dế mèn phiê")
print(result_of_gru_char)
result_of_gru_char = generate_text(generate_model, start_string=u"nhà kh")
print(result_of_gru_char)
result_of_gru_char = generate_text(generate_model, start_string=u"sách tập làm v")
print(result_of_gru_char)
result_of_gru_char = generate_text(generate_model, start_string=u"thanh lọ")
print(result_of_gru_char)

(1, 11)
dế mèn phiêu lưu ký tái nhà ăn cơm học
(1, 6)
nhà khi đúng b
(1, 14)
sách tập làm việc nhà thuật x
(1, 8)
thanh lọc ốc diệu của philập tư duy vệ sách mẹ nhà trường chứng khoán nhật kỳ lực chi kháng kèm s


## **4.2. LSTM**

In [68]:
result_of_gru_char = generate_text(generate_model_lstm, start_string=u"dế mèn phiê")
print(result_of_gru_char)
result_of_gru_char = generate_text(generate_model_lstm, start_string=u"nhà kh")
print(result_of_gru_char)
result_of_gru_char = generate_text(generate_model_lstm, start_string=u"sách tập làm v")
print(result_of_gru_char)
result_of_gru_char = generate_text(generate_model_lstm, start_string=u"thanh lọ")
print(result_of_gru_char)

(1, 11)
dế mèn phiêu lưu ký khi những điều lấp lánh được gọi tên tái bản
(1, 6)
nhà khoa học
(1, 14)
sách tập làm văn
(1, 8)
thanh lọc cơ thể và giảm cân


# **5. Spelling correction**

## **5.1. Left to Right without Lookahead**

In [206]:
def correct_text(model, text, begin=7, threshold=0.001):
  correct = text[:begin]
  misspell = text[:begin]
  misspell_detected = False

  print("Assume the first " + str(begin) + " chars are correct")
  seq = [char2idx[c] for c in text[:begin]]
  seq = tf.expand_dims(seq, 0)
  model.reset_states()

  for i in range(begin, len(text)):
    predictions = model(seq)
    predictions = tf.squeeze(predictions, 0)[-1]
    probs = special.softmax(predictions)

    if probs[char2idx[text[i]]] < threshold:
      misspell_detected = True
      misspell += "(" + text[i] + ")"
      corrected_char = tf.math.top_k(predictions).indices[0]
      correct += idx2char[corrected_char]
      print(f"{misspell} --> {correct}")
    else:
      misspell += text[i]
      correct += text[i]

    seq = tf.expand_dims([char2idx[correct[-1]]], 0)

  if not misspell_detected:
    misspell = ""
  
  print("misspell: ", misspell)
  print("correct: ", correct)
  print()
  return correct, misspell

In [207]:
# Good cases
correct_text(generate_model_lstm, "dế mèn phieu lưu ký táo bản")
correct_text(generate_model_lstm, "dòng suoi nguồn thịnh vuong")
correct_text(generate_model_lstm, "dòng suối nguồn thịnh vượng")
print()

Assume the first 7 chars are correct
dế mèn phi(e) --> dế mèn phiê
dế mèn phi(e)u lưu ký tá(o) --> dế mèn phiêu lưu ký tái
misspell:  dế mèn phi(e)u lưu ký tá(o) bản
correct:  dế mèn phiêu lưu ký tái bản

Assume the first 7 chars are correct
dòng su(o) --> dòng suố
dòng su(o)i nguồn thịnh v(u) --> dòng suối nguồn thịnh vư
dòng su(o)i nguồn thịnh v(u)(o) --> dòng suối nguồn thịnh vượ
misspell:  dòng su(o)i nguồn thịnh v(u)(o)ng
correct:  dòng suối nguồn thịnh vượng

Assume the first 7 chars are correct
misspell:  
correct:  dòng suối nguồn thịnh vượng




In [224]:
# bad case
correct_text(generate_model_lstm, "dòng suối nnguồn thịnh vượng")
correct_text(generate_model_lstm, "dòng suối naaguồn thịnh vượng")
print()

Assume the first 7 chars are correct
dòng suối n(n) --> dòng suối ng
dòng suối n(n)(g) --> dòng suối ngu
dòng suối n(n)(g)(u) --> dòng suối nguồ
dòng suối n(n)(g)(u)(ồ) --> dòng suối nguồn
dòng suối n(n)(g)(u)(ồ)(n) --> dòng suối nguồn 
dòng suối n(n)(g)(u)(ồ)(n)( ) --> dòng suối nguồn t
dòng suối n(n)(g)(u)(ồ)(n)( )(t) --> dòng suối nguồn th
dòng suối n(n)(g)(u)(ồ)(n)( )(t)(h) --> dòng suối nguồn thị
dòng suối n(n)(g)(u)(ồ)(n)( )(t)(h)(ị) --> dòng suối nguồn thịn
dòng suối n(n)(g)(u)(ồ)(n)( )(t)(h)(ị)(n) --> dòng suối nguồn thịnh
dòng suối n(n)(g)(u)(ồ)(n)( )(t)(h)(ị)(n)(h) --> dòng suối nguồn thịnh 
dòng suối n(n)(g)(u)(ồ)(n)( )(t)(h)(ị)(n)(h)( ) --> dòng suối nguồn thịnh v
dòng suối n(n)(g)(u)(ồ)(n)( )(t)(h)(ị)(n)(h)( )(v) --> dòng suối nguồn thịnh vư
dòng suối n(n)(g)(u)(ồ)(n)( )(t)(h)(ị)(n)(h)( )(v)(ư) --> dòng suối nguồn thịnh vượ
dòng suối n(n)(g)(u)(ồ)(n)( )(t)(h)(ị)(n)(h)( )(v)(ư)(ợ) --> dòng suối nguồn thịnh vượn
dòng suối n(n)(g)(u)(ồ)(n)( )(t)(h)(ị)(n)(h)( )(v)(ư)(ợ)(n) -->

## **5.2 Left to Right with Lookahead**

In [197]:
def get_prob_of_text(model, text, begin):
  prob = 1
  if begin >= len(text):
    return prob
  
  seq = [char2idx[c] for c in text]
  model.reset_states()
  predictions = model(tf.expand_dims(seq, 0))
  predictions = tf.squeeze(predictions, 0)
  for i in range(begin, len(text)):
    probs = special.softmax(predictions[i-1])
    prob *= probs[char2idx[text[i]]]

  return prob

test = ["dế mèn phiêu lưu ký", "dế mèn phiêu lu ký", "dế mèn phiêu ưu ký"]
for t in test:
  print(t, ":", get_prob_of_text(generate_model_lstm, t, 10))

dế mèn phiêu lưu ký : 0.8976038268369191
dế mèn phiêu lu ký : 3.850490837557988e-06
dế mèn phiêu ưu ký : 2.5335562370135195e-07


In [211]:
def correct_text_lookahead(model, text, begin=7, threshold=0.001):
  correct = text[:begin]
  misspell = text[:begin]
  misspell_detected = False

  print("Assume the first " + str(begin) + " chars are correct")

  seq = [char2idx[c] for c in text[:begin]]
  for i in range(begin, len(text)):
    model.reset_states()
    predictions = model(tf.expand_dims(seq, 0))
    predictions = tf.squeeze(predictions, 0)[-1]
    probs = special.softmax(predictions)

    if probs[char2idx[text[i]]] < threshold:
      misspell_detected = True
      top_k_next_chars = tf.math.top_k(probs, k=3).indices
      options = [correct + idx2char[c] + text[i+1:] for c in top_k_next_chars] # replace text[i]
      options.append(correct + text[i+1:]) # remove text[i]
      options_probs = [get_prob_of_text(model, option, len(correct)) for option in options]
      chosen = np.argmax(options_probs)
      misspell += "(" + text[i] + ")"
      if chosen != len(options)-1:
        corrected_char = top_k_next_chars[chosen]
        correct += idx2char[corrected_char]
      print(f"{misspell} --> {correct}")
    else:
      misspell += text[i]
      correct += text[i]

    seq.append(char2idx[correct[-1]])

  if not misspell_detected:
    misspell = ""
  
  print(f"Misspell: {misspell}\nCorrect: {correct}\n")
  return correct, misspell

In [226]:
correct_text_lookahead(generate_model_lstm, "dế mèn phieu lưu ký táo bản")
correct_text_lookahead(generate_model_lstm, "dòng suoi nguồn thịnh vuợng")
correct_text_lookahead(generate_model_lstm, "dòng suối nguồn thịnh vượng")

Assume the first 7 chars are correct
dế mèn phi(e) --> dế mèn phiê
dế mèn phi(e)u lưu ký tá(o) --> dế mèn phiêu lưu ký tái
Misspell: dế mèn phi(e)u lưu ký tá(o) bản
Correct: dế mèn phiêu lưu ký tái bản

Assume the first 7 chars are correct
dòng su(o) --> dòng suố
dòng su(o)i nguồn thịnh v(u) --> dòng suối nguồn thịnh vư
Misspell: dòng su(o)i nguồn thịnh v(u)ợng
Correct: dòng suối nguồn thịnh vượng

Assume the first 7 chars are correct
Misspell: 
Correct: dòng suối nguồn thịnh vượng



('dòng suối nguồn thịnh vượng', '')

In [223]:
correct_text_lookahead(generate_model_lstm, "dòng suối nnguồn thịnh vượng")
correct_text_lookahead(generate_model_lstm, "dòng suối naaguồn thịnh vượng")
print()

Assume the first 7 chars are correct
dòng suối n(n) --> dòng suối n
Misspell: dòng suối n(n)guồn thịnh vượng
Correct: dòng suối nguồn thịnh vượng

Assume the first 7 chars are correct
dòng suối n(a) --> dòng suối n
dòng suối n(a)(a) --> dòng suối n
Misspell: dòng suối n(a)(a)guồn thịnh vượng
Correct: dòng suối nguồn thịnh vượng


